In [23]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from path import Path
import gc
import optuna
from sklearn.model_selection import StratifiedKFold
from scipy.special import erfinv

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Input, BatchNormalization,Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation, LeakyReLU

get_custom_objects().update({'leaky-relu':Activation(LeakyReLU(alpha  = 0.2))})


In [5]:
def gpu_cleanup(objects):
    if objects:
        del(objects)
    K.clear_session()
    gc.collect()

In [42]:
class Config:
    input_path = Path('../')
    dae_batch_size = 128
    dae_num_epoch = 50
    dae_architecture = [1500, 1500,1500]
    reuse_autoencoder = False
    batch_size = 128
    num_epocs = 150
    units = [64, 32]
    input_dropout = 0.06
    dropout = 0.08
    regL2 = 0.09
    activation = 'relu'

    cv_folds = 5
    nas = True
    random_state = 0

config = Config()

In [17]:
train = pd.read_csv(config.input_path/'input/train.csv', index_col= 'id')
test = pd.read_csv(config.input_path/'input/test.csv', index_col= 'id')
submission = pd.read_csv(config.input_path/'input/sample_submission.csv', index_col= 'id')

In [18]:
calc_features = [feat for feat in train.columns if "_calc" in feat]
cat_features = [feat for feat in train.columns if "_cat" in feat]
target = train["target"]
train = train.drop("target", axis="columns")
train = train.drop(calc_features, axis="columns")
test = test.drop(calc_features, axis="columns")
train = pd.get_dummies(train, columns=cat_features)
test = pd.get_dummies(test, columns=cat_features)


In [19]:
print("applying gaussrank to columns: ", end = '')
to_normalize = list()

for k, col in enumerate(train.columns):
    if '_bin' not in col and '_cat' not in col and '_missing' not in col:
        to_normalize.append(col)
print(to_normalize)

def to_gauss(x): return np.sqrt(2) * erfinv(x)
def normalize(data, norm_cols):
    n = data.shape[0]
    for col in norm_cols:
        sorted_idx = data[col].sort_values().index.tolist()
        uniform = np.linspace(start = -0.99, stop = 0.99, num = n)
        normal = to_gauss(uniform)
        normalized_col = pd.Series(index = sorted_idx, data = normal)
        data[col] = normalized_col
    return data

train = normalize(train, to_normalize)
test = normalize(test, to_normalize)

applying gaussrank to columns: ['ps_ind_01', 'ps_ind_03', 'ps_ind_14', 'ps_ind_15', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15']


In [20]:
features = train.columns
train_index = train.index
test_index = test.index
train = train.values.astype(np.float32)
test = test.values.astype(np.float32)

In [24]:
def plot_keras_history(history, measures):
    rows = len(measures)//2 + len(measures) % 2
    fig, panels = plt.subplots(rows, 2, figsize = (15,5))
    plt.subplots_adjust(top = 0.99, bottom = 0.01, hspace = 0.4, wspace = 0.2)

    try:
        panels = [item for sublist in panels for item in sublist]

    except:
        pass

    for k, measure in enumerate(measures):
        panel = panels[k]
        panel.set_title(measure + 'history')
        panel.plot(history.epoch, history.history['measure'], label = 'train '+measure)

        try:
            panel.plot(history.epoch,
                       history.history['val_'+measure],
                       label = 'validation '+measure)
        except:
            pass

        panel.set(xlabel = 'epochs', ylabel = measure)
        panel.legend()

    plt.show(fig)


from numba import jit
@jit
def eval_gini(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_pred)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

    

C:\Users\fuksy\AppData\Local\Temp\ipykernel_3156\2068861814.py:31: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [ ]:
def batch_generator(x, batch_size, shuffle = True, random_state = None):
    
    batch_index = 0 #initial batch starts at reference 0
    n = x.shape[0] #total length of the data that is going generate the batchs

    while True: #keep going
        if batch_index == 0: #first run
            index_array = np.arange(n) #setting the ordered index of the data 
            
            if shuffle: #in case of shuffle, set the seed and shuffle the index
                np.random.seed(seed = random_state)
                index_array = np.random.permutation(n) #permutation has something similar to np.arange when used in integers

            current_index = (batch_index * batch_size) % n #the current index is going to drop only when surprass n
            if n>= current_index + batch_size: # if its not, then keep updating the batch_index
                current_batch_size = batch_size
                batch_index += 1

            else: #when the actual batch surpass the number of data
                current_batch_size = n - current_index #how many points are overpassing n
                batch_index = 0 #return to the first batch
            
            batch = x[index_array[current_index:current_index+current_batch_size]] #

            yield batch

In [25]:
def mixup_generator(X, batch_size, swaprate=0.15, shuffle=True, random_state=None):
    if random_state is None:
        random_state = np.randint(0, 999)
    num_features = X.shape[1]
    num_swaps = int(num_features * swaprate)    
    generator_a = batch_generator(X, batch_size, shuffle, 
                                  random_state)
    generator_b = batch_generator(X, batch_size, shuffle, 
                                  random_state + 1)
    while True:
        batch = next(generator_a)
        mixed_batch = batch.copy()
        effective_batch_size = batch.shape[0]
        alternative_batch = next(generator_b)
        assert((batch != alternative_batch).any())
        for i in range(effective_batch_size):
            swap_idx = np.random.choice(num_features, num_swaps, 
                                        replace=False)
            mixed_batch[i, swap_idx] = alternative_batch[i, swap_idx]
        yield (mixed_batch, batch)



In [26]:
def get_DAE(X, architecture = [1500,1500,1500]):
    features = X.shape[1]
    inputs = Input((features,))

    for i, nodes in enumerate(architecture):
        layer = Dense(nodes,
                       activation = 'relu',
                       use_bias = False,
                       name = f'code_{i+1}')
        
        if i == 0:
            x = layer(inputs)
        else:
            x = layer(x)

        x = BatchNormalization()(x)
    outputs = Dense(features,activation = 'linear')(x)
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mse','mae'])
    return model

In [30]:
def extract_dae_features(autoencoder, X, layers = [3], batch_size = 128):
    data = []
    for layer in layers:
        if layer == 0:
            data.append(X)
        else:
            get_layer_output = Model([autoencoder.layers[0].input],
                                     [autoencoder.layers[layer].output])
            
            layer_output = get_layer_output.predict(X, batch_size = batch_size)

            data.append(layer_output)
        data = np.hstack(data)
        return data


In [31]:
def autoencoder_fit(X_train, X_valid, filename = 'dae',random_state = None, suppress_output = False):
    if suppress_output:
        verbose = 0
    else:
        verbose = 2
        print('fitting a denoise autoencoder')
        
    tf.random.set_seed(seed = random_state)

    generator = mixup_generator(X_train,
                                batch_size=config.batch_size,
                                swaprate=0.15,
                                random_state=config.random_state)
    
    dae = get_DAE(X_train, architecture=config.dae_architecture)

    steps_per_epoch = np.ceil(X_train.shape[0]/config.dae_batch_size)

    early_stopping = EarlyStopping(monitor = 'val_mse',
                                   mode = 'min',
                                   patience = 5,
                                   restore_best_weigths = True,
                                   verbose = 0)
    
    history = dae.fit(generator,
                      steps_per_epoch = steps_per_epoch,
                      epochs = config.num_epocs, 
                      validation_data = (X_valid, X_valid),
                      verbose = verbose)

    if not suppress_output:
        plot_keras_history(history, measures = ['mse','mae'])

    dae.save(filename)
    return dae


In [32]:
def dense_blocks(x, units, activation, regL2, dropout):
    kernel_initializer = keras.initializers.RandomNormal(mean = 0.0,
                                                         stddev = 0.01, seed = config.random_state)
    
    for k,layer_units in enumerate(units):
        if regL2 >0:
            x = Dense(layer_units, activation=activation,
                      kernel_initializer = kernel_initializer,
                      kernel_regularizer = l2(regL2))(x)
            
        else:
            x = Dense(layers_units, activation = activation,
                      kernel_initializer = kernel_initializer)(x)
            
        if dropout > 0:
            x = Dropout(dropout)(x)

        return x

In [34]:
def dnn_model(dae, units = [4500,1000,1000],
              input_dropout = 0.1, dropout = 0.5,
              regL2 = 0.05,
              activation = 'relu'):
    
    inputs = dae.get_layer('code_2').output
    
    if input_dropout > 0 :
        x = Dropout(input_dropout)(inputs)

    else:
        x = tf.keras.layers.Layer()(inputs)
        
    x = dense_blocks(x, units, activation, regL2, dropout)
    outputs = Dense(1, activation = 'sigmoid')(x)

    model = Model(inputs = dae.input, outputs = outputs)    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001),
                  loss = keras.losses.binary_crossentropy,
                  metrics = [AUC(name = 'auc')])
    
    return model

In [35]:
def model_fitting(X_train, y_train, X_valid, y_valid,
                  autoencoder,
                  filename,
                  random_state = None,
                  suppress_output = False):
    
    if suppress_output:
        verbose = 0

    else:
        print('fitting model')
        verbose = 2
    
    early_stopping = EarlyStopping(monitor='val_auc',
                                   mode = 'max',
                                   patience = 10,
                                   restore_best_weights = True,
                                   verbose = 0)
    
    rlrop = ReduceLROnPlateau(monitor = 'val_auc',
                              mode = 'max',
                              patience = 2,
                              factor = 0.75,
                              verbose = 0)
    
    tf.random.set_seed(seed = random_state)
    model = dnn_model(autoencoder,
                      units = config.units,
                      input_dropout=config.input_dropout,
                      dropout=config.dropout,
                      regL2 = config.regL2,
                      activation=config.activation)
    
    history = model.fit(X_train,y_train,
                        epochs = config.num_epocs,
                        batch_size = config.batch_size,
                        validation_data = (X_valid, y_valid),
                        callbacks = [early_stopping, rlrop],
                        shuffle = True,
                        verbose = verbose)
    model.save(filename)

    if not suppress_output:
        plot_keras_history(history, measures = ['loss','auc'])

    return model,history

In [45]:
if config.nas is True:
    def evaluate():
        metric_evaluations = list()
        skf = StratifiedKFold(n_splits = config.cv_folds,
                              shuffle=True,
                              random_state=config.random_state)
        
        for k, (train_idx, valid_idx) in enumerate(skf.split(train,target)):
            X_train, y_train = train[train_idx,:], target[train_idx]
            X_valid, y_valid = train[valid_idx,:], target[valid_idx]

            if config.reuse_autoencoder:
                autoencoder = load_model(f"./dae_fold{k}")
            else:
                autoencoder = autoencoder_fit(X_train,X_valid,
                                              filename= f'./dae_fold{k}',
                                              random_state=config.random_state,
                                              suppress_output=True)
                
            model,_ = model_fitting(X_train,y_train, X_valid,y_valid,
                                    autoencoder=autoencoder,
                                    filename = f'dnn_model_fold{k}',
                                    random_state = config.random_state,
                                    suppress_output = True)
            
            val_preds = model.predict(X_valid, batch_size = 128, verbose = 0)
            best_score = eval_gini(y_true = y_valid, y_pred = np.ravel(val_preds))
            metric_evaluations.append(best_score)
            gpu_cleanup([autoencoder, model])

        return np.mean(metric_evaluations)

In [46]:
def objective(trial):
    params = {'first_layer':trial.suggest_categorical('first_layer',[8,16,32,64,128,256,512]),
              'second_layer':trial.suggest_categorical('second_layer',[0,8,16,32,64,128,256]),
              'third_layer':trial.suggest_categorical('third_layer',[0,8,16,32,64,128,256]),
              'input_drop':trial.suggest_float('input_droput',0.0, 0.5),
              'dropout':trial.suggest_float('dropout', 0.0, 0.5),
              'regL2':trial.suggest_float('regL2',0.0, 0.1),
              'activation':trial.suggest_categorical('activation',['relu','leaky-relu','selu'])}
    
    config.units = [nodes for nodes in [params['first_layer'],
                                        params['second_layer'],
                                        params['third_layer']] if nodes >0 ]
    
    config.input_dropout = params['input_drop']
    config.dropout = params['dropout']
    config.regL2 = params['regL2']
    config.activation = params['activation']

    return evaluate()

study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials=60)
print('best gini score', study.best_value)
print('best params', study.best_params)




[I 2023-07-20 08:53:17,764] A new study created in memory with name: no-name-91761593-6d1d-47b6-8102-f3eeb02413e7
[W 2023-07-20 08:53:18,123] Trial 0 failed with parameters: {'first_layer': 128, 'second_layer': 0, 'third_layer': 32, 'input_droput': 0.41397418263980235, 'dropout': 0.32289256322728677, 'regL2': 0.077476909525998, 'activation': 'selu'} because of the following error: ValueError('too many values to unpack (expected 2)').
Traceback (most recent call last):
  File "c:\Users\fuksy\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\fuksy\AppData\Local\Temp\ipykernel_3156\1475215444.py", line 19, in objective
    return evaluate()
           ^^^^^^^^^^
  File "C:\Users\fuksy\AppData\Local\Temp\ipykernel_3156\1472530446.py", line 9, in evaluate
    X_train, y_train = train[train_idx,:]; target[train_idx]
    ^^^^^^^^^^^^^^^^
ValueError: too many values to unpack (exp

ValueError: too many values to unpack (expected 2)

In [ ]:
config.units = [nodes for nodes in [study.best_params['first_layer'],
                                     study.best_params['second_layer'],
                                       study.best_params['third_layer']] if nodes > 0]

config.input_dropout = study.best_params['input_dropout']
    config.dropout = study.best_params['dropout']
    config.regL2 = study.best_params['regL2']
    config.activation = study.best_params['activation']

